# Estudio de datos sobre nuestros amigos peludos

## Import de librerias

In [ ]:
import pandas as pd
import numpy as np
import altair as alt

## montado de drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Carga de data

In [ ]:
dogs_data = pd.read_csv("/content/drive/MyDrive/Dogs_Project/best_in_show.csv")

## Limpiando el dataset de columnas vacias y sacando la info adicional del conjunto de datos.

In [ ]:
dogs_data = dogs_data.loc[:,~dogs_data.columns.str.match("Unnamed")]


In [ ]:
additional_info = pd.DataFrame(dogs_data[:][0: 1])
additional_info

,Dog breed,category,datadog score,POPULARITY IN US,POPULARITY IN US.1,"LIFETIME COST, $",5 LIFETIME COST,1 INTELLIGENCE (TRAINABILITY) ranking,INTELLIGENCE (TRAINABILITY) ranking,2 LONGEVITY,...,"food per lifetime, $","Other regular costs, total per lifetime, $","total per year, $","total, per year, £","toys, presents, treats, per year, £","pet sitters, per year, £","grooming, per year, £","vet fees per year, £","kennels per year, £","one offs, $"
0,Additional info,American Kennel Club group,NaN,"1- 173, 1 = most poular, US. American Kennel C...",ranking within breeds with full data only,NaN,"in descending rank (higher score = better), n...",1 = easiest to train / learn new commands,"in descending rank (higher score = better), n...","years, weighted average - see note",...,food per week x 52.1775 x average lifespan (co...,"if no average lifespan data, then 'no data'",converted to US $ using Google Finance,NaN,averaged across all dog breeds. pet food exclu...,NaN,NaN,NaN,NaN,"Initial outlay: leash, collar, ID tag; food di..."


In [ ]:
dogs_data = dogs_data[:][1:]

## Analisis inicial

Instalando pandas profilling

In [ ]:
import sys
!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

Ejecucion y visualizacion del reporte

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(dogs_data, title="Dogs Report")

In [ ]:
profile

Output hidden; open in https://colab.research.google.com to view.

# Analisis extenso

## Cual es el top 5 de perros mas populares en USA?

In [ ]:
no_nan_data = dogs_data[~(dogs_data["POPULARITY IN US.1"].isin([np.nan, 'NaN']))]
no_nan_data["POPULARITY IN US.1"] = no_nan_data["POPULARITY IN US.1"].astype('int64')
no_nan_data.sort_values("POPULARITY IN US.1")[["Dog breed", "POPULARITY IN US.1"]].head(5)

<ipython-input-17-cba32dfb3705>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_nan_data["POPULARITY IN US.1"] = no_nan_data["POPULARITY IN US.1"].astype('int64')


,Dog breed,POPULARITY IN US.1
27,Labrador Retriever,1
69,German Shepherd,2
38,Beagle,3
36,Golden Retriever,4
32,Yorkshire Terrier,5


## cual es el promedio de vida de los perros entre todas sus razas?

reemplazo de datos no data por NaN

In [ ]:
dogs_data["2 LONGEVITY"] = dogs_data["2 LONGEVITY"].replace("no data", np.nan)

In [ ]:
dogs_data["2 LONGEVITY"]

1      12.52
2      14.00
3      12.92
4      13.84
5      12.49
       ...  
170      NaN
171      NaN
172    13.17
173     8.80
174      NaN
Name: 2 LONGEVITY, Length: 174, dtype: object

arreglo de dato extraño

In [ ]:
dogs_data = dogs_data.replace("1.83 - really?", 1.83)

conversion de tipo de la columna

In [ ]:
dogs_data["2 LONGEVITY"]  = dogs_data["2 LONGEVITY"].astype("float64")

### Visualizacion de distribucion

como se puede observar la distrbucion de longevidad entre razas es mas o menos uniforme, excepto por el black russian Terrier, es un poco raro este registro ya que en otros datasets el promedio de vida de esta raza esta entre los 10 - 11 años.



In [ ]:
alt.Chart(dogs_data).mark_bar().encode(
    y="2 LONGEVITY",
    x= "Dog breed"
)

alt.Chart(...)

modificacion de dato anomalo

In [ ]:
dogs_data["2 LONGEVITY"] = dogs_data["2 LONGEVITY"].replace(1.83, 11)

### el promedio de edad seria de aproximadamente 11 años

In [ ]:
dogs_data["2 LONGEVITY"].mean()

10.95705882352941

### hay una correlacion entre el tamaño y longevidad de los perros?

In [ ]:
alt.Chart(dogs_data).mark_point().encode(
    x='weight (kg):Q',
    y='2 LONGEVITY',
    color='size category'
)

alt.Chart(...)

Aunque es leve si se logra ver una pequeña correlacion negativa en cuanto al tamaño/peso de un perro con su longevidad, podria decir que es es mas probable que viva mas un perro pequeño a uno grande.

## Cuales son los perros mas baratos de mantener?

Extraigo la data que no es nula

In [ ]:
no_nan_data = dogs_data[~(dogs_data["food per lifetime, $"] == "no data") & ~(dogs_data["Other regular costs, total per lifetime, $"] == "no data") & ~(dogs_data["Dog breed"].isin(["NOT POSSIBLE IF NO INTELLIGENCE DATA", "Looked - nothing"]))]

In [ ]:
no_nan_data

,Dog breed,category,datadog score,POPULARITY IN US,POPULARITY IN US.1,"LIFETIME COST, $",5 LIFETIME COST,1 INTELLIGENCE (TRAINABILITY) ranking,INTELLIGENCE (TRAINABILITY) ranking,2 LONGEVITY,...,"food per lifetime, $","Other regular costs, total per lifetime, $","total per year, $","total, per year, £","toys, presents, treats, per year, £","pet sitters, per year, £","grooming, per year, £","vet fees per year, £","kennels per year, £","one offs, $"
1,Border Collie,herding,3.64,45,39,"$20,143",48%,1,100%,12.52,...,"3,486","13,095","1,046",784.0,121,126.0,244.0,177.0,116.0,200
2,Border Terrier,terrier,3.61,80,61,"$22,638",14%,30,70%,14.00,...,"3,898","14,643","1,046",784.0,121,126.0,244.0,177.0,116.0,200
3,Brittany,sporting,3.54,30,30,"$22,589",16%,19,80%,12.92,...,"5,171","13,514","1,046",784.0,121,126.0,244.0,177.0,116.0,200
4,Cairn Terrier,terrier,3.53,59,48,"$21,992",22%,35,61%,13.84,...,"3,854","14,476","1,046",784.0,121,126.0,244.0,177.0,116.0,200
5,Welsh Springer Spaniel,sporting,3.34,130,81,"$20,224",47%,31,69%,12.49,...,"3,478","13,064","1,046",784.0,121,126.0,244.0,177.0,116.0,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Otterhound,hound,NaN,169,NaN,no data,NaN,46,NaN,10.80,...,"4,323","11,296","1,046",784.0,121,126.0,244.0,177.0,116.0,200
150,Polish Lowland Sheepdog,herding,NaN,159,NaN,no data,NaN,no data,NaN,10.80,...,"4,323","11,296","1,046",784.0,121,126.0,244.0,177.0,116.0,200
165,Sussex Spaniel,sporting,NaN,161,NaN,no data,NaN,no data,NaN,11.17,...,"4,471","11,683","1,046",784.0,121,126.0,244.0,177.0,116.0,200
166,Swedish Vallhund,herding,NaN,153,NaN,"$22,839",NaN,no data,NaN,14.17,...,"3,946","14,821","1,046",784.0,121,126.0,244.0,177.0,116.0,200


creo un df solo de los gastos en comida

In [ ]:
gasto_comida_vida = no_nan_data[["Dog breed", "food per lifetime, $"]].rename(columns={"food per lifetime, $" : "valor en dolares", "Dog breed" : "raza"})
gasto_comida_vida["tipo de gasto"] = "comida"
gasto_comida_vida

,raza,valor en dolares,tipo de gasto
1,Border Collie,"3,486",comida
2,Border Terrier,"3,898",comida
3,Brittany,"5,171",comida
4,Cairn Terrier,"3,854",comida
5,Welsh Springer Spaniel,"3,478",comida
...,...,...,...
146,Otterhound,"4,323",comida
150,Polish Lowland Sheepdog,"4,323",comida
165,Sussex Spaniel,"4,471",comida
166,Swedish Vallhund,"3,946",comida


creo otro df con los otros gastos

In [ ]:
gasto_varios_vida = no_nan_data[["Dog breed", "Other regular costs, total per lifetime, $"]].rename(columns={"Other regular costs, total per lifetime, $" : "valor en dolares", "Dog breed" : "raza"})
gasto_varios_vida["tipo de gasto"] = "gastos varios como peluqueria, veterinaria y juguetes"
gasto_varios_vida

,raza,valor en dolares,tipo de gasto
1,Border Collie,"13,095","gastos varios como peluqueria, veterinaria y j..."
2,Border Terrier,"14,643","gastos varios como peluqueria, veterinaria y j..."
3,Brittany,"13,514","gastos varios como peluqueria, veterinaria y j..."
4,Cairn Terrier,"14,476","gastos varios como peluqueria, veterinaria y j..."
5,Welsh Springer Spaniel,"13,064","gastos varios como peluqueria, veterinaria y j..."
...,...,...,...
146,Otterhound,"11,296","gastos varios como peluqueria, veterinaria y j..."
150,Polish Lowland Sheepdog,"11,296","gastos varios como peluqueria, veterinaria y j..."
165,Sussex Spaniel,"11,683","gastos varios como peluqueria, veterinaria y j..."
166,Swedish Vallhund,"14,821","gastos varios como peluqueria, veterinaria y j..."


uno los 2

In [ ]:
spends_per_life_df = pd.concat([gasto_comida_vida, gasto_varios_vida], ignore_index=True)

quito las comas en los valores

In [ ]:
spends_per_life_df["valor en dolares"] = spends_per_life_df["valor en dolares"].apply(lambda x:str(x).replace(",", ""))

In [ ]:
spends_per_life_df

,raza,valor en dolares,tipo de gasto
0,Border Collie,3486,comida
1,Border Terrier,3898,comida
2,Brittany,5171,comida
3,Cairn Terrier,3854,comida
4,Welsh Springer Spaniel,3478,comida
...,...,...,...
201,Otterhound,11296,"gastos varios como peluqueria, veterinaria y j..."
202,Polish Lowland Sheepdog,11296,"gastos varios como peluqueria, veterinaria y j..."
203,Sussex Spaniel,11683,"gastos varios como peluqueria, veterinaria y j..."
204,Swedish Vallhund,14821,"gastos varios como peluqueria, veterinaria y j..."


Visualizo

In [ ]:
alt.Chart(spends_per_life_df).mark_bar().encode(
    x=alt.X('raza:N', sort='y', title="Raza"),
    y=alt.Y('valor en dolares:Q', title="Valor total promedio gastado en dolares a lo largo de la vida del perro"),
    color='tipo de gasto:N',
    tooltip=["valor en dolares", "tipo de gasto"]
).interactive()

alt.Chart(...)

Aqui podemos ver una grafica del promedio de gasto en dolares atravez de toda la vida de un perro dividido por razas y tipo de gasto.

entonces para responder la pregunta del principio podria colocar un grupo conformado por 8 perros los cuales sus gastos no pasan de los 12.000 dolares estarian los bulldog, los nova scotia, los mastiff,  los finnish laphund, y los bullmastiff, los bloodhound, los great dane y los leonberger y dentro de estos un subgrupo los cuales no sobrepasan los 10.000 dolares los cuales serian el bulldog y el nova scotia

con esta grafica tambien podria responder otra pergunta.

## Cuales son los perros mas caros de mantener?

aqui podria enumerar un grupo de 7 los cuales sus gastos superan los 20000 dolares en promedio a travez de toda su vida, los germand shorthhaired pointer, giant schnauzer, pointer, samoyed, tibetan spaniel, canaan dog, chihuhua

## Pequeña curiosidad

una pequeña curiosida es que es comun que el gasto en comida de un perro sea un cuarto del total de gastos pero en algunos perros este gasto sobrepasa ese cuarto y se aproxima a la mitad total de los gastos, estos son el irish wolfhound, saint bernard y giant schnauzer.
Acaso se debe al gran tamaño de estos y por eso comen mas? o acaso tienen mejor salud y se enferman menos? lo que bajaria el porcentaje de gasto en salud.
quisera saber las correlaciones de esto pero no estoy seguro de tener los datos.

## El tamaño de un perro influye en su intelgencia/entrenabilidad?

limpio los registros que no tienen data

In [ ]:
dogs_data_without_nan_weight = dogs_data[~(dogs_data["weight (kg)"].isin(["no data", "NA (3 classes)", np.nan])) & ~(dogs_data["INTELLIGENCE (TRAINABILITY) ranking"].isin([np.nan, "NaN"]))]
dogs_data_without_nan_weight

,Dog breed,category,datadog score,POPULARITY IN US,POPULARITY IN US.1,"LIFETIME COST, $",5 LIFETIME COST,1 INTELLIGENCE (TRAINABILITY) ranking,INTELLIGENCE (TRAINABILITY) ranking,2 LONGEVITY,...,"food per lifetime, $","Other regular costs, total per lifetime, $","total per year, $","total, per year, £","toys, presents, treats, per year, £","pet sitters, per year, £","grooming, per year, £","vet fees per year, £","kennels per year, £","one offs, $"
2,Border Terrier,terrier,3.61,80,61,"$22,638",14%,30,70%,14.00,...,"3,898","14,643","1,046",784.0,121,126.0,244.0,177.0,116.0,200
3,Brittany,sporting,3.54,30,30,"$22,589",16%,19,80%,12.92,...,"5,171","13,514","1,046",784.0,121,126.0,244.0,177.0,116.0,200
4,Cairn Terrier,terrier,3.53,59,48,"$21,992",22%,35,61%,13.84,...,"3,854","14,476","1,046",784.0,121,126.0,244.0,177.0,116.0,200
6,English Cocker Spaniel,sporting,3.33,63,51,"$18,993",63%,18,82%,11.66,...,"3,247","12,196","1,046",784.0,121,126.0,244.0,177.0,116.0,200
7,Cocker Spaniel,sporting,3.30,27,27,"$24,330",9%,20,79%,12.50,...,"6,149","13,074","1,046",784.0,121,126.0,244.0,177.0,116.0,200
10,Shetland Sheepdog,herding,3.22,20,20,"$21,006",34%,6,94%,12.53,...,"3,698","13,106","1,046",784.0,121,126.0,244.0,177.0,116.0,200
11,Siberian Husky,working,3.22,16,16,"$22,049",20%,45,45%,12.58,...,"5,035","13,158","1,046",784.0,121,126.0,244.0,177.0,116.0,200
12,Lhasa Apso,non-sporting,3.21,62,50,"$22,031",21%,68,15%,13.92,...,"3,876","14,560","1,046",784.0,121,126.0,244.0,177.0,116.0,200
14,Dachshund,hound,3.19,9,9,"$20,113",49%,49,37%,12.63,...,"3,517","13,210","1,046",784.0,121,126.0,244.0,177.0,116.0,200
15,Miniature Schnauzer,terrier,3.19,12,12,"$20,087",51%,12,87%,11.81,...,"3,486","12,353","1,046",784.0,121,126.0,244.0,177.0,116.0,200


In [ ]:
dogs_data_without_nan_weight[["weight (kg)","INTELLIGENCE (TRAINABILITY) ranking"]]

,weight (kg),INTELLIGENCE (TRAINABILITY) ranking
2,6,70
3,16,80
4,6,61
6,14,82
7,11,79
10,10.15,94
11,22,45
12,7,15
14,11,37
15,7,87


Me deshago del simbolo de porcentaje

In [ ]:
dogs_data_without_nan_weight["INTELLIGENCE (TRAINABILITY) ranking"] = dogs_data_without_nan_weight["INTELLIGENCE (TRAINABILITY) ranking"].apply(lambda x: x.replace("%", ""))

<ipython-input-148-29b753ce7afd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dogs_data_without_nan_weight["INTELLIGENCE (TRAINABILITY) ranking"] = dogs_data_without_nan_weight["INTELLIGENCE (TRAINABILITY) ranking"].apply(lambda x: x.replace("%", ""))


In [ ]:
alt.Chart(dogs_data_without_nan_weight).mark_circle().encode(
    x=alt.X('weight (kg):Q', title="Peso en kg"),
    y=alt.Y('INTELLIGENCE (TRAINABILITY) ranking:Q', title="Porcentaje de facilidad al aprender trucos"),
    color=alt.Color('size category:O', title="Tamaño", scale=alt.Scale(range=['yellow','orange','red'])),
    tooltip=["Dog breed", "weight (kg)"]
).interactive()

alt.Chart(...)

no veo una correlacion evidente entre el tamaño y la entrenabilidad en verdad, hay tanto perros grandes, medianos y pequeños que son faciles o dificiles de entrenar, podria decir que talvez los perros extremadamente grandes son mas tontos o mas dificiles de entrenar que perros con tamaños medianos o pequeños?



para seguir indagando con la curiosidad de mas arriba, voy a investigar mas sobre la siguiente pregunta.

## Cuales pueden desarrollar mas o menos enfermedades geneticas?

limpiando donde no hay data y aquellos perros con 0 enfermedades geneticas registradas

In [ ]:
no_nan_diseases = dogs_data[~(dogs_data['3 NO. OF GENETIC AILMENTS'].isin(["no data", np.nan]) & ~(dogs_data['3 NO. OF GENETIC AILMENTS'] == '0'))]

In [ ]:
no_nan_diseases = no_nan_diseases.rename(columns={"3 NO. OF GENETIC AILMENTS" : "Número de posibles enfermedades geneticas"})

In [ ]:
no_nan_diseases = no_nan_diseases[~(no_nan_diseases['Número de posibles enfermedades geneticas'] == '0')]

In [ ]:
no_nan_diseases[['Dog breed','Número de posibles enfermedades geneticas']]

,Dog breed,Número de posibles enfermedades geneticas
1,Border Collie,2
4,Cairn Terrier,2
5,Welsh Springer Spaniel,1
7,Cocker Spaniel,2
8,Papillon,5
...,...,...
156,Scottish Deerhound,2
157,Sealyham Terrier,1
158,Shiba Inu,1
164,Standard Schnauzer,1


In [ ]:
alt.Chart(no_nan_diseases).mark_bar().encode(
    y=alt.Y('Número de posibles enfermedades geneticas:Q'),
    x=alt.X("Dog breed:N", sort="-y", title='Raza'),
    color="size category:N"
)

alt.Chart(...)

Parece ser que el top 5 de perros con mas posibles enfermedades geneticas son, el bulldog, shetland springer spaniel, papilom, german sheherd, pembroke welsh corgi, dentro de este grupo los ultimos 2 tienen muchas mas posibles enfermedades que los demas.

ahora con esto quiero saber.
## como se divide la cantidad de enfermedades entre los diferentes tipos de tamaño?

cambio el nombre de las variables para que el grafico quede mejor

In [ ]:
no_nan_diseases['size category'] = no_nan_diseases['size category'].replace(to_replace=["large", "medium", "small"], value=['Grande', 'Mediano', 'Pequeño'])

agrupo los datos por categoria de tamaño

In [ ]:
size_groups = no_nan_diseases.groupby('size category')

Obtengo la suma total de posibles enfermedades geneticas segun cada tipo de tamaño

In [ ]:
big_dogs_diseases = size_groups.get_group('Grande')['Número de posibles enfermedades geneticas'].astype('int64').sum()

In [ ]:
medium_dogs_diseases = size_groups.get_group('Mediano')['Número de posibles enfermedades geneticas'].astype('int64').sum()

In [ ]:
small_dogs_diseases = size_groups.get_group('Pequeño')['Número de posibles enfermedades geneticas'].astype('int64').sum()

In [ ]:
total_number_of_diseases = big_dogs_diseases + medium_dogs_diseases + small_dogs_diseases

 creo un nuevo dataframe con la distribucion de la data como la quiero

In [ ]:
diseases_by_size = pd.DataFrame({"Tamaño" : ['Grande', 'Mediano', 'Pequeño'], 'Número de posibles enfermedades geneticas' : [big_dogs_diseases, medium_dogs_diseases, small_dogs_diseases] })

In [ ]:
print(round(big_dogs_diseases / total_number_of_diseases * 100))
print(round(medium_dogs_diseases / total_number_of_diseases * 100))
print(round(small_dogs_diseases / total_number_of_diseases * 100))

40
30
30


In [ ]:
alt.Chart(diseases_by_size).mark_arc().encode(
    theta=alt.Theta(field="Número de posibles enfermedades geneticas", type="quantitative"),
    color=alt.Color(field="Tamaño", type="nominal", scale=alt.Scale(range=["#42113C", "#618B25", '#6BD425'])),
)

alt.Chart(...)

quisiera agregar porcentajes desde altair pero por mas que intente y busque no hallar una manera factible de colocar labels con el porcentaje de cada grupo desde altair, ademas de ello este grafico puede tener problemas debido a que puede que hayan emfermedades repetidas entre los grupos lo que haga que aumente su cantidad, aunque como estan repetidas en todos los grupos creo que se refelja bien la distribucion de enfermedades entre los diferentes tamaños, por ultimo parece ser que hay un poco mas de enfermedades geneticas posibles en perros grandes?